# RecoMIDI AI (ver. 1.0)

## Dead simple and super-efficient MIDI recommendation implementation

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (Setup Environment)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/RecoMIDI
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import tqdm as tqdm

print('Loading TMIDIX module...')
os.chdir('/content/RecoMIDI')

import TMIDIX

from GPT2RGAX import *

# (Unzip and prep the model)

In [ ]:
#@title Unzip pre-trained model


%cd /content/RecoMIDI/Model/

print('=' * 70)
print('Unzipping the model...Please wait...')
print('=' * 70)

!cat RecoMIDI-Trained-Model.zip* > RecoMIDI-Trained-Model.zip
print('=' * 70)

!unzip -j RecoMIDI-Trained-Model.zip
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/RecoMIDI/

In [ ]:
#@title Load/Reload the model

DIC_SIZE=9300

from collections import OrderedDict

full_path_to_model_checkpoint = "/content/RecoMIDI/Model/RecoMIDI-Trained-Model.pth" #@param {type:"string"}

print('Loading the model...')
config = GPTConfig(DIC_SIZE, 
                   1024,
                   dim_feedforward=512,
                   n_layer=6, 
                   n_head=8, 
                   n_embd=512,
                   enable_rpr=True,
                   er_len=1024)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GPT(config)

state_dict = torch.load(full_path_to_model_checkpoint, map_location=device)

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

model.to(device)

model.eval()

print('Done!')

# (LAKH/clean_midi Signatures Pack)

In [ ]:
#@title Load LAKH/clean_midi Signatures Pack
artists, songs, artists_dic, songs_dic, sigs, INTS = pickle.load(open('/content/RecoMIDI/RecoMIDIAI.pickle', 'rb'))

# (GENERATE MATCHES)

In [ ]:
#@title Generate
full_path_to_source_MIDI = "/content/RecoMIDI/Come To My Window.mid" #@param {type:"string"}
f = full_path_to_source_MIDI

#print('Loading MIDI file...')
score = TMIDIX.midi2score(open(f, 'rb').read())
inputs = []
events_matrix = []
inputs_sigs = []
itrack = 1

while itrack < len(score):
    for event in score[itrack]:         
        if event[0] == 'note':
            events_matrix.append(event)
    itrack += 1
events_matrix.sort()
if len(events_matrix) > 0:
    instruments_pitches = [y[4] for y in events_matrix if y[3] != 9]
    instruments_pitches_count = len(instruments_pitches)
    instruments_pitches_sum = sum(instruments_pitches)

    drums_pitches = [y[4] for y in events_matrix if y[3] == 9]
    drums_pitches_count = len(drums_pitches)
    drums_pitches_sum = sum(drums_pitches)

    inputs.append([f, '', 
                            instruments_pitches_count, instruments_pitches_sum,
                           drums_pitches_count, drums_pitches_sum, events_matrix[-1][1]])

    
inputs_sigs.extend([inputs[0][2] % 512, inputs[0][2] // 512,
                inputs[0][3] % 512, inputs[0][3] // 512,
                inputs[0][4] % 512, inputs[0][4] // 512,
                inputs[0][5] % 512, inputs[0][5] // 512,
                inputs[0][6] % 512])


temperature = 1

rand_seq = model.generate(torch.Tensor(inputs_sigs), 
                                            target_seq_length=len(inputs_sigs)+2,
                                            temperature=temperature,
                                            stop_token=DIC_SIZE,
                                            verbose=True)
    
out = rand_seq[0].cpu().numpy().tolist()

print(out)

print(artists_dic[out[-2]])

print(songs_dic[out[-1]])

# Congrats! You did it! :)